In [78]:
import numpy as np
import pandas as pd
import plotly.express as px
import nixio
from usz_neuro_conversion.common import read_nix, NixContext

In [79]:
nix_context = NixContext(
    session=1, subject=1, project="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM"
)
nixfile = read_nix(nix_context)
# nixfile.pprint()

_general = nixfile.sections["General"]
recording_location = _general.props["Recording location"].values[0]
recording_setup = (
    _general.sections["Recording setup"].props["Recording setup EEG"].values[0]
)

_electrode_map = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG electrode information"]
    .data_arrays["Scalp_Electrode_EEGLAB_BESA_Coordinates"]
)

# See <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
# in x, y ,z
eeg_electrode_locations = np.ndarray(_electrode_map.shape)
_electrode_map.read_direct(eeg_electrode_locations)

In [81]:
print(recording_location)
print(recording_setup)
# print(eeg_electrode_locations)
print(_general.sections["Related publications"].props)

Schweizerische Epilepsie-Zentrum, 8008 Zurich, Switzerland
NicOne; original sampling rate, 256 Hz; passband, 0.3 to 100 Hz
[Property: {name = Publication name}, Property: {name = Publication DOI}]
a


In [ ]:
# BESA map: +X is nose, +Y is left, +Z is up according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
eeg_electrode_locations_df = pd.DataFrame(
    eeg_electrode_locations, columns=["X", "Y", "Z"]
)
# By eye: ["F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2", "F7", "F8", "T7", "T8", "P7", "P8", "FZ", "CZ", "PZ", "LPA", "RPA"]
_labels = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)
eeg_electrode_locations_df.insert(3, "location", _labels)
print(eeg_electrode_locations_df.to_string())
fig = px.scatter_3d(
    eeg_electrode_locations_df,
    x="X",
    y="Y",
    z="Z",
    color=eeg_electrode_locations_df.index,
)
fig.show()

               X             Y          Z location
0   5.755106e+01  4.820043e+01  39.869712       F3
1   5.758403e+01 -4.814260e+01  39.891983       F4
2   3.868125e-15  6.317128e+01  56.871691       C3
3   3.867882e-15 -6.316731e+01  56.876102       C4
4  -5.755106e+01  4.820043e+01  39.869712       P3
5  -5.758403e+01 -4.814260e+01  39.891983       P4
6  -8.078401e+01  2.613301e+01  -4.001085       O1
7  -8.078401e+01 -2.613301e+01  -4.001085       O2
8   4.987138e+01  6.842334e+01  -7.489518       F7
9   4.992653e+01 -6.838359e+01  -7.485085       F8
10  5.176493e-15  8.453854e+01  -8.845083       T3
11  5.176493e-15 -8.453854e+01  -8.845083       T4
12 -4.987138e+01  6.842334e+01  -7.489518       T5
13 -4.992653e+01 -6.838359e+01  -7.485085       T6
14  6.073848e+01 -0.000000e+00  59.462904       Fz
15  5.204749e-15 -0.000000e+00  85.000000       Cz
16 -6.073848e+01 -7.438319e-15  59.462904       Pz
17  3.680313e-15  6.010408e+01 -60.104076       A1
18  3.680313e-15 -6.010408e+01 

In [ ]:
_sampled_dimension = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[1]
)
print(f"Start at {_sampled_dimension.offset} {_sampled_dimension.unit} w.r.t. fixation")
print(f"Sampled every {_sampled_dimension.sampling_interval} {_sampled_dimension.unit}")

_events = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
)
_EVENTS_PER_TRIAL = 5
trials = 1
for i in range(0, trials * _EVENTS_PER_TRIAL, _EVENTS_PER_TRIAL):
    print("--")
    for j in range(_EVENTS_PER_TRIAL):
        _event = _events[i + j]
        print(
            f"Event {_event.name}: {_event.position[1]} {_event.units[0]} (w.r.t. fixation), lasted {_event.extent[1]} {_event.units[0]}"
        )

In [ ]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[1]
)

In [ ]:
ref = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)
print(ref)
for i in range(1, 50):
    assert (
            nixfile.blocks["Data_Subject_01_Session_01"]
            .groups["Scalp EEG data"]
            .data_arrays[f"Scalp_EEG_Data_Trial_{i:02d}"]
            .dimensions[0]
            .labels
            == ref
    )
print("All trials have the same electrode order")

In [ ]:
ieeg_electrode_information = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays
)
print(ieeg_electrode_information)
print(ieeg_electrode_information["iEEG_Electrode_Map"][:])
print(
    ieeg_electrode_information["iEEG_Electrode_Map"].sources["iEEG electrode 5"].sources
)
print(ieeg_electrode_information["iEEG_Electrode_MNI_Coordinates"].shape)
print(ieeg_electrode_information["iEEG_Electrode_Manual_Entry"].dimensions[0].labels)
print(
    list(
        zip(
            ieeg_electrode_information["iEEG_Electrode_Map"][:],
            ieeg_electrode_information["iEEG_Electrode_Manual_Entry"]
            .dimensions[0]
            .labels,
        )
    )
)

In [ ]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike times"].data_arrays)

In [ ]:
print(nixfile.sections["Session"].sections["Trial properties"].sections)

In [ ]:
print(nixfile.sections["Subject"].props["Age"].name)
print(nixfile.sections["Subject"].props["Age"].values[0])

In [ ]:
print(
    len(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["iEEG electrode information"]
        .data_arrays["iEEG_Electrode_Manual_Entry"]
        .dimensions[0]
        .labels
    )
)

In [ ]:
print(
    len(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Scalp EEG data"]
        .data_arrays["Scalp_EEG_Data_Trial_01"]
        .dimensions[0]
        .labels
    )
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)

In [ ]:
print(
    nixfile.sections["Session"].sections["Trial properties"].sections["Trial_01"].props
)
print(
    nixfile.sections["Session"]
    .sections["Trial properties"]
    .sections["Trial_01"]
    .props["Response time"]
    .values[0]
)

In [ ]:
tag = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags[3]
)
print(tag.name, tag.position, tag.units, tag.extent)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"][0]
)

In [82]:
[
    (tag.name, tag.position, tag.extent)
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
]

[('Event_Fixation_ss8_c1_m2_Trial_01_Scalp_EEG',
  (0.0, -6.0),
  (9.4e-323, 5e-324)),
 ('Event_Stimulus_ss8_xxxxxxxx_Trial_01_Scalp_EEG',
  (0.0, -5.0),
  (9.4e-323, 1e-323)),
 ('Event_Maintenance_Trial_01_Scalp_EEG', (0.0, -3.0), (9.4e-323, 1.5e-323)),
 ('Event_Probe_m2_H_Trial_01_Scalp_EEG', (0.0, 0.0), (9.4e-323, 0.0)),
 ('Event_Response_c1_Trial_01_Scalp_EEG', (0.0, 2.484), (9.4e-323, 0.0)),
 ('Event_Fixation_ss4_c1_m1_Trial_02_Scalp_EEG',
  (0.0, -6.0),
  (9.4e-323, 5e-324)),
 ('Event_Stimulus_ss4_xxxxxxxx_Trial_02_Scalp_EEG',
  (0.0, -5.0),
  (9.4e-323, 1e-323)),
 ('Event_Maintenance_Trial_02_Scalp_EEG', (0.0, -3.0), (9.4e-323, 1.5e-323)),
 ('Event_Probe_m1_T_Trial_02_Scalp_EEG', (0.0, 0.0), (9.4e-323, 0.0)),
 ('Event_Response_c1_Trial_02_Scalp_EEG', (0.0, 1.66775), (9.4e-323, 0.0)),
 ('Event_Fixation_ss8_c1_m2_Trial_03_Scalp_EEG',
  (0.0, -6.0),
  (9.4e-323, 5e-324)),
 ('Event_Stimulus_ss8_xxxxxxxx_Trial_03_Scalp_EEG',
  (0.0, -5.0),
  (9.4e-323, 1e-323)),
 ('Event_Maintenance_

In [ ]:
[
    (tag.name, tag.position, tag.extent)
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags iEEG"]
    .tags
]

In [ ]:
sum(
    [
        (a.position[1] - b.position[1])
        for (a, b) in zip(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Trial events single tags iEEG"]
        .tags,
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Trial events single tags scalp EEG"]
        .tags,
    )
    ]
)

In [ ]:
print(
    tag[:]
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
)

In [ ]:
[
    (
        trial.props["Probe letter"].values[0],
        trial.props["Match"].values[0],
        trial.props["Response"].values[0],
        bool(trial.props["Correct"].values[0]),
    )
    for trial in nixfile.sections["Session"].sections["Trial properties"].sections
]

In [ ]:
[
    trial.dimensions[1].sampling_interval * trial.shape[1] + trial.dimensions[1].offset
    for trial in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays
]

In [ ]:
[
    (tag.name, tag.position, tag.extent)
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags spike times"]
    .tags
]

In [ ]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups)

In [ ]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays[0]
    .shape
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays[0][:]
)
print(
    len(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Spike waveforms"]
        .data_arrays
    )
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike waveforms"].data_arrays
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays[0]
    .dimensions[1]
    .sampling_interval
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Manual_Entry"]
    .dimensions[0]
    .labels
)

In [ ]:
print(
    [
        times.name
        for times in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike times"]
    .data_arrays
    ]
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike times"].data_arrays[0][:]
)

In [ ]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike times multitags"]
    .multi_tags[0]
    .features[0]
    .data[0]
)

In [ ]:
nixfile.sections["Session"].props["Trial duration"].values[0]

In [ ]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Map"]
    .sources[0]
    .sources[0]
    .name
)

# Amygdala

In [87]:
nix_context = NixContext(
    session=1, subject=1, project="Human_Amygdala_MUA_sEEG_FearVideo"
)
nixfile = read_nix(nix_context)

In [ ]:
print([(trial.props["Condition"].values[0], trial.props["Condition name"].values[0]) for trial in
       nixfile.sections["Session"].sections["Trial properties"].sections])

In [ ]:
print([channel.sources[2].name
       for channel in nixfile.blocks["Data_Subject_01_Session_01"].groups["iEEG electrode information"]
      .data_arrays["iEEG_Electrode_Map"]
      .sources])

In [ ]:
spike_events = nixfile.blocks["Data_Subject_01_Session_01"].groups["Trial events single tags spike times"].tags
ieeg_events = nixfile.blocks["Data_Subject_01_Session_01"].groups["Trial events single tags iEEG"].tags
[(spike_event.name, ieeg_event.name) for spike_event, ieeg_event in zip(spike_events, ieeg_events)]

In [96]:
print([spike.name for spike in nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike waveforms"].data_arrays])

['Spike_Waveform_Unit_1_uAL_1', 'Spike_Waveform_Unit_2_uAL_2', 'Spike_Waveform_Unit_3_uAL_6', 'Spike_Waveform_Unit_4_uAR_7']


In [ ]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups["Trial events single tags iEEG"].tags[4].position)

In [98]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups["iEEG electrode information"].data_arrays[
          "iEEG_Electrode_Map"].sources[0].sources[0].name,
      nixfile.blocks["Data_Subject_01_Session_01"].groups["iEEG electrode information"].data_arrays[
          "iEEG_Electrode_Map"].sources[1].sources[0].name)

mAL2-mAL1 mAR2-mAR1


# Visual WM

In [156]:
nix_context = NixContext(
    session=1, subject=1, project="Human_MTL_units_visual_WM",
)
nixfile = read_nix(nix_context)

In [157]:
print([block.name for block in nixfile.blocks])
print([section.name for section in nixfile.sections])

['Data_Subject_02_Session_01']
['General', 'Task', 'Subject', 'Session']


In [158]:
print([section.name for section in nixfile.sections["Session"].props])
print([section.name for section in nixfile.sections["Session"].sections])
print([prop.name for prop in nixfile.sections["Subject"].props])
print([section.name for section in nixfile.sections["Subject"].sections])
print([prop.name for prop in nixfile.sections["Task"].props])
print([section.name for section in nixfile.sections["Task"].sections])
print([prop.name for prop in nixfile.sections["General"].props])
print([section.name for section in nixfile.sections["General"].sections])

['Number of trials', 'Trial duration']
['Trial properties']
['Age', 'Sex', 'Pathology', 'Handedness', 'Implanted electrodes', 'Electrodes in seizure onset zone (SOZ)']
[]
['Task name', 'Task description']
[]
['Institution', 'Recording location']
['Related publications', 'Recording setup']


In [159]:

print([(prop.name, prop.values[0]) for prop in
       nixfile.sections["Session"].sections["Trial properties"].sections[0].props])

[('Trial number', 1.0), ('Set size', 2.0), ('Match', 1.0), ('Correct', 1.0), ('Response', 1.0), ('Response time', 0.5160000000000001)]


In [160]:
print(
    [prop.name for prop in
     nixfile.sections["General"].sections["Recording setup"].props])
print(nixfile.sections["General"].sections["Recording setup"].props["Recording setup iEEG"].values[0])

['Recording setup iEEG']
Neuralynx ATLAS; original sampling rate, 32 kHz; passband, 1 to 8000 Hz


In [162]:
print([group.name for group in nixfile.blocks["Data_Subject_01_Session_01"].groups])
print("\n".join([f"{tag.name}: {tag.position}, {tag.extent}, {tag.sources}" for tag in
                 nixfile.blocks["Data_Subject_01_Session_01"].groups["Trial events single tags spike times"].tags]))

['Trial events single tags spike times']


KeyError: "Item not found 'Data_Subject_01_Session_01'"

In [153]:
print([group.name for group in nixfile.blocks["Data_Subject_01_Session_01"].groups])


['Trial events single tags spike times']
